In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from datetime import datetime
import os
import random
import pandas as pd

In [2]:
!mkdir output

A subdirectory or file output already exists.


In [13]:
def multi_brownian_motion(num_particles, num_steps, radius, m=0.01, c=0.05, xy_range=(0, 100)):
    sigmas = m * np.array(radius) + c
    
    positions = np.zeros((num_steps, num_particles, 2))
    positions[0, :, 0] = np.random.uniform(xy_range[0], xy_range[1], num_particles)
    positions[0, :, 1] = np.random.uniform(xy_range[0], xy_range[1], num_particles)
    
    for step in range(1, num_steps):
        move = np.column_stack((np.random.normal(0, sigmas, num_particles), 
                                 np.random.normal(0, sigmas, num_particles)))
        positions[step] = positions[step - 1] + move
    return positions

## Multi Particle Brownian Motion Simulation

In [3]:
def multi_brownian_motion(num_particles, num_steps, sigma=0.08, xy_range=(0, 100)):
    positions = np.zeros((num_steps, num_particles, 2))
    positions[0, :, 0] = np.random.uniform(xy_range[0], xy_range[1], num_particles)
    positions[0, :, 1] = np.random.uniform(xy_range[0], xy_range[1], num_particles)
    
    for step in range(1, num_steps):
        positions[step] = positions[step - 1] + np.random.normal(0, sigma, (num_particles, 2))
    return positions

## Model Parameters

### Figure is in 100x100 Cartesian coordinate system where the bottom-left corner is designated as (0,0) and the top-right corner is designated as (100,100)

In [9]:
num_particles = 500
num_steps = 100

resolution = (2048, 2048)

#in pixels
radius = np.random.normal(5, 1, num_particles)
radius = abs(radius.round(4))

grayscale_value = [random.randint(50, 255) for _ in range(num_particles)]
fill_color = [(g, g, g) for g in grayscale_value]

positions = multi_brownian_motion(num_particles, num_steps, radius, xy_range=(1, 1999))

## New Image Generation

In [15]:
current_date_and_time = datetime.now()
folder = current_date_and_time.strftime('%Y-%m-%d-%H-%M-%S')
os.makedirs(f'output/{folder}', exist_ok=True)
for i in range(num_steps):
    img = Image.new('RGB', resolution, color="black")
    draw = ImageDraw.Draw(img)

    for idx, (x, y) in enumerate(positions[i]):
        # x = x*resolution[0]/100
        # y = y*resolution[1]/100
        r = radius[idx]

        left_up_point = (x-r, y-r)
        right_down_point = (x+r, y+r)
        try:
            draw.ellipse([left_up_point, right_down_point], fill=fill_color[idx])
        except:
            print(idx)
            print(r)
            print(left_up_point)
            print(right_down_point)
    filename = f"{i+1}.png"
    img.save(f'output/{folder}/{filename}')
    #print(f"Image saved as {'output/' + folder + '/' + filename}")

## Save Coordinates

In [16]:
dic = {}
for i in range(num_steps):
    l = []
    for j in range(num_particles):
        l.append(positions[i][j])
    dic[f'Step {i+1}'] = l

df = pd.DataFrame.from_dict(dic, orient='index').T

In [12]:
df.to_csv(f'output/{folder}/{folder}_coordinates.csv')

In [17]:
df

,Step 1,Step 2,Step 3,Step 4,Step 5,Step 6,Step 7,Step 8,Step 9,Step 10,...,Step 91,Step 92,Step 93,Step 94,Step 95,Step 96,Step 97,Step 98,Step 99,Step 100
0,"[916.8952898415681, 516.4460281939167]","[916.8536386911695, 516.3992691209316]","[916.8439490580771, 516.4516555301318]","[916.811649649763, 516.5135302871495]","[916.8084413149212, 516.4178353648543]","[916.5732342435733, 516.4524779307218]","[916.515778029549, 516.347818743393]","[916.6099876261334, 516.3623708682911]","[916.6244658711908, 516.2869344364445]","[916.6681638346772, 516.1283843050915]",...,"[916.9464564824855, 516.1146238657826]","[916.9297205751012, 515.9959265442442]","[916.8875186245938, 515.8849967529344]","[916.8549592221758, 515.8708950567452]","[916.8896754196344, 515.8946440922449]","[916.783024544329, 515.8449650611608]","[916.7323445164064, 515.7485486066201]","[916.7520441645479, 515.6772894513733]","[916.8031114888246, 515.4967069023969]","[916.8464585900658, 515.4032070502786]"
1,"[1537.575308101076, 884.0517336634767]","[1537.6275638580817, 884.0861906662612]","[1537.5945679611182, 883.9965633734475]","[1537.6210575671523, 884.0475947837531]","[1537.6830951720262, 883.8862362235958]","[1537.6993922166182, 883.8600284737598]","[1537.7228991379684, 883.8154353982611]","[1537.7154870356383, 884.0126503643479]","[1537.7162669366269, 884.1749903674031]","[1537.6525418945505, 884.3215782055424]",...,"[1537.8251987259393, 883.3332731397318]","[1537.8003241694194, 883.4937735296646]","[1537.9538831860682, 883.3821543166966]","[1537.9781361103387, 883.2704512637001]","[1538.0780709196865, 883.2373103827423]","[1537.9949294881585, 883.2549581884815]","[1538.2012782029499, 883.3465453046185]","[1538.1740737344328, 883.2346107482537]","[1538.1257772351805, 883.1275288143223]","[1538.225174733696, 883.163380861511]"
2,"[1102.8746938275688, 1846.6427405606842]","[1102.681582306909, 1846.5186044544478]","[1102.5307966126713, 1846.4449689116886]","[1102.7323770920996, 1846.42286778194]","[1102.6867876763902, 1846.3731480759945]","[1102.7384608625193, 1846.413089844991]","[1102.6980946331726, 1846.4282204381384]","[1102.7527060456423, 1846.2490557203362]","[1102.695533632468, 1846.1127986793997]","[1102.6778825024016, 1846.1975425644127]",...,"[1103.62803527878, 1845.1042191917263]","[1103.6231549531362, 1844.9289196104228]","[1103.6876791007646, 1844.819025703617]","[1103.5772158902316, 1844.8108010694875]","[1103.4515107372524, 1844.9910561026152]","[1103.2792755491107, 1845.1849090681435]","[1103.2932280789244, 1845.105750117518]","[1103.146475508187, 1845.3166557241937]","[1103.1974993171011, 1845.351481774612]","[1103.2509068259121, 1845.3226352757772]"
3,"[1692.2185554646078, 1281.0524311010026]","[1692.1691627604787, 1281.1100866842614]","[1692.2052078060622, 1281.1797228404916]","[1692.2741990849568, 1281.2531207807006]","[1692.4083644383377, 1281.1741958710336]","[1692.4403010767458, 1281.123773075752]","[1692.522005409866, 1281.1155158633665]","[1692.5539034155063, 1281.184050960238]","[1692.3864774056121, 1281.1388858948774]","[1692.4499232510113, 1281.0473068873291]",...,"[1693.5729767080084, 1279.4801913487058]","[1693.751366819518, 1279.5103050211017]","[1693.83907408312, 1279.6429071849539]","[1693.8963583538705, 1279.5732321735386]","[1693.884100964556, 1279.7108422818328]","[1693.9877989486217, 1279.824062247585]","[1693.9979078316578, 1279.690605153543]","[1693.9210034106043, 1279.7617888593343]","[1694.0171743782496, 1279.7619369637396]","[1693.9812440051178, 1279.860639704335]"
4,"[1451.586975849847, 1809.0494024999434]","[1451.596876767058, 1809.240629729242]","[1451.7409552906693, 1809.1551586358648]","[1451.659687397417, 1809.230364205583]","[1451.736171700569, 1809.3359032599053]","[1451.8499480672674, 1809.4519960227676]","[1451.7897078276308, 1809.5427884532673]","[1451.8537098435, 1809.5040461164908]","[1451.8034323344398, 1809.4694773319522]","[1451.9034402043485, 1809.7137129545888]",...,"[1452.2585736366523, 1809.110891691556]

## Old Image Generation

In [8]:
# Old Version
# dpi= 80

# for i in range(len(positions)):
#   fig, ax = plt.subplots(figsize=(resolution[0]/dpi,resolution[1]/dpi))
#   coordinates = positions[i]
#   x = coordinates[:, 0]
#   y = coordinates[:, 1]
#   ax.scatter(x, y, sizes, color='white', marker='o')
#   ax.set_xlim(0, 100)
#   ax.set_ylim(0, 100)
#   ax.set_aspect('equal')
#   ax.axis('off')
#   fig.patch.set_facecolor("black")
#   fig.tight_layout()
#   fig.savefig(f'output/{i+1}o.png', bbox_inches='tight', pad_inches=0, dpi=resolution[0]/40)

## Turn series of images as video

In [9]:
import cv2
import os

def get_numeric_part(filename):
    return int(filename.split('.')[0])

def images_to_video(image_folder, video_name, fps=24):
    images = [img for img in os.listdir(image_folder) if img.endswith(".png")] 
    images = sorted(images, key=get_numeric_part)
    
    frame = cv2.imread(os.path.join(image_folder, images[0]))
    h, w, layers = frame.shape
    size = (w, h)

    out = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'MP4V'), fps, size)  # 'DIVX' is a codec used to compress and decompress video files, you can use 'XVID' or 'MP4V' for .avi or .mp4 format respectively
    
    for image in images:
        img_path = os.path.join(image_folder, image)
        img = cv2.imread(img_path)
        out.write(img)
    
    out.release()


images_to_video(f'output/{folder}', f'output/{folder}/{folder}_output_video.mp4')